In [24]:
!pip install git+https://github.com/ichironakamoto/Japan.git

  Cloning https://github.com/ichironakamoto/Japan.git to /tmp/pip-req-build-7cvwsq66
  Running command git clone -q https://github.com/ichironakamoto/Japan.git /tmp/pip-req-build-7cvwsq66
  Created wheel for covid19-inference: filename=covid19_inference-0.0.10-cp36-none-any.whl size=27625 sha256=5c14ec8823f49315c184367e3306a3582c1442355e0c452a8538a590177dc5be
  Stored in directory: /tmp/pip-ephem-wheel-cache-fcrb_z4f/wheels/1e/a5/16/7d019c629951d7e8b52c76c794e547a19333680b3c6dbac3bd
Successfully built covid19-inference


In [0]:
import datetime
import time as time_module
import sys
import os 
import pickle

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import theano
import matplotlib
import pymc3 as pm
import theano.tensor as tt



try: 
    import covid19_inference as cov19
except ModuleNotFoundError:
    sys.path.append('../..')
    import covid19_inference as cov19

path_to_save = '../../figures/'
path_save_pickled = '../../data/'
rerun = True

In [26]:

confirmed_cases = cov19.get_jhu_confirmed_cases()

country = 'Germany'
date_data_begin = datetime.datetime(2020,3,1)
# date_data_end   = cov19.get_last_date(confirmed_cases)
date_data_end = datetime.datetime(2020,4,21)

#date_data_end   = datetime.datetime(2020,3,28)
num_days_data = (date_data_end-date_data_begin).days
diff_data_sim = 16 # should be significantly larger than the expected delay, in 
                   # order to always fit the same number of data points.
num_days_future = 28
date_begin_sim = date_data_begin - datetime.timedelta(days = diff_data_sim)
date_end_sim   = date_data_end   + datetime.timedelta(days = num_days_future)
num_days_sim = (date_end_sim-date_begin_sim).days


cases_obs = cov19.filter_one_country(confirmed_cases, country,
                                     date_data_begin, date_data_end)

print('Cases yesterday ({}): {} and '
      'day before yesterday: {}'.format(date_data_end.isoformat(), *cases_obs[:-3:-1]))

prior_date_mild_dist_begin =  datetime.datetime(2020,3,9)
prior_date_strong_dist_begin =  datetime.datetime(2020,3,16)
prior_date_contact_ban_begin =  datetime.datetime(2020,3,23)

change_points = [dict(pr_mean_date_begin_transient = prior_date_mild_dist_begin,
                      pr_sigma_date_begin_transient = 3,
                      pr_median_lambda = 0.2,
                      pr_sigma_lambda = 0.5),
                 dict(pr_mean_date_begin_transient = prior_date_strong_dist_begin,
                      pr_sigma_date_begin_transient = 1,
                      pr_median_lambda = 1/8,
                      pr_sigma_lambda = 0.5),
                 dict(pr_mean_date_begin_transient = prior_date_contact_ban_begin,
                      pr_sigma_date_begin_transient = 1,
                      pr_median_lambda = 1/8/2,
                      pr_sigma_lambda = 0.5)]

start = pm.find_MAP()
step = pm.Metropolis()


if rerun:

    traces = []
    models = []

    for num_change_points in range(4):
        model = cov19.SIR_with_change_points(new_cases_obs = np.diff(cases_obs),
                                            change_points_list = change_points[:num_change_points],
                                            date_begin_simulation = date_begin_sim,
                                            num_days_sim = num_days_sim,
                                            diff_data_sim = diff_data_sim,
                                            N = 83e6,
                                            priors_dict=None,
                                            weekends_modulated=False)
        models.append(model)
        traces.append(pm.sample(model=model, init='advi', n_init=6000,draws=4000, step=step, start=start))


    pickle.dump([models, traces], open(path_save_pickled + 'SIR_without_sine2.pickled', 'wb'))

else: 
    models, traces = pickle.load(open(path_save_pickled + 'SIR_without_sine2.pickled', 'rb'))

    

Cases yesterday (2020-04-21T00:00:00): 148291 and day before yesterday: 147065


/usr/local/lib/python3.6/dist-packages/pymc3/tuning/starting.py:61: UserWarning: find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.
  warnings.warn('find_MAP should not be used to initialize the NUTS sampler, simply call pymc3.sample() and it will automatically initialize NUTS in a better way.')


TypeError: ignored

In [0]:
exec(open('figures_revised_old_layout.py').read())


In [0]:
trace = traces[0]
fig, ax = plt.subplots(figsize=(5,4))
time = np.arange(-len(cases_obs)+1, 0)
mpl_dates = conv_time_to_mpl_dates(time) 
ax.plot(mpl_dates, np.abs(np.median(trace.new_cases[:, :num_days_data], axis=0) - np.diff(cases_obs)), 
        'd', markersize=6,
         label='Absolute difference\n'
               'between fit and data')
ax.plot(mpl_dates, np.sqrt(np.median(trace.new_cases[:, :num_days_data], axis=0))*np.median(trace.sigma_obs, axis=0),
         label='Width of the likelihood', lw=3)
ax.set_ylabel('Difference (number of new cases)')
ax.set_xlabel('Date')
ax.legend(loc='upper left')
print(np.median(np.sum(trace.new_cases[:, :num_days_data], axis=1)+ trace.I_begin))
#plt.tight_layout()
ax.xaxis.set_major_locator(matplotlib.dates.AutoDateLocator())
ax.xaxis.set_major_formatter(matplotlib.dates.DateFormatter('%-m/%-d'))


In [0]:
print(len(models), len(traces))
create_figure_distributions(models[3], traces[3],
                              additional_insets = None, xlim_lambda = (0, 0.53), color = 'tab:cyan',
                              num_changepoints=3, xlim_tbegin=7, save_to = path_to_save +'Fig_SIR_without_sine_weekend_dist')


In [0]:
create_figure_timeseries(traces[3], 'tab:cyan',
                       plot_red_axis=True, save_to = path_to_save +'Fig_SIR_without_sine_weekend_cases')


In [0]:
print('\n0 step model\n')
print(pm.loo(traces[0], models[0]))

print('\n1 step model\n')
print(pm.loo(traces[1], models[1]))

print('\n2 steps model\n')
print(pm.loo(traces[2], models[2]))

print('\n3 steps model\n')
print(pm.loo(traces[3], models[3]))

In [0]:
for j in range(4):
    print(f'lambda* {j}')   
    print(print_median_CI(traces[0][f"lambda_{j}"] - traces[0].mu, prec=2))